In [ ]:
!git clone https://github.com/MeesLursen/KernelLTL.git

In [ ]:
!cd /kaggle/working/KernelLTL && git pull && cd ..

In [ ]:
import sys
sys.path.insert(0, "/kaggle/working/KernelLTL") 

In [ ]:
!wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
!./bin/micromamba create -y -n kernelltl -c conda-forge python=3.13.7 pip
!./bin/micromamba run -n kernelltl pip install -r /kaggle/working/KernelLTL/requirements.txt

In [ ]:
!cd /kaggle/working/KernelLTL/LTL
!./bin/micromamba run -n kernelltl torchrun --nproc_per_node=2 -m LTL.train